In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
import numpy as np
import pandas as pd
import cv2
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation, BatchNormalization
import tensorflow as tf

In [ ]:
print(os.listdir("/gdrive/My Drive/DerinOgrenmeProje/lung_image_sets"))

['lung_squamous cell carcinoma', 'lung_adenocarcinoma', 'lung_benign tissue']


In [ ]:
DATADIR = '/gdrive/My Drive/DerinOgrenmeProje/lung_image_sets'
CATEGORIES = ['lung_squamous cell carcinoma', 'lung_adenocarcinoma', 'lung_benign tissue']

In [ ]:
IMG_SIZE = 100

In [ ]:
all_data =[]

def create_all_data():
    for category in CATEGORIES:
        path=os.path.join(DATADIR, category)
        class_num=CATEGORIES.index(category)
        for img in os.listdir(path):
            try:
                img_array=cv2.imread(os.path.join(path,img))
                new_array=cv2.resize(img_array,(IMG_SIZE,IMG_SIZE))
                all_data.append([new_array,class_num])
            except Exception as e:
                pass
create_all_data()

In [ ]:
print("Toplam Görüntü Sayısı: ", len(all_data))

Toplam Görüntü Sayısı:  15000


In [ ]:
X = []
y = []

for categories, label in all_data:
  X.append(categories)
  y.append(label)

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state =42)

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.10, random_state=42)

In [ ]:
x_train = np.array(x_train)
x_test = np.array(x_test)

y_train = np.array(y_train)
y_test = np.array(y_test)

x_val = np.array(x_val)
y_val = np.array(y_val)

In [ ]:
train_yCl = tf.keras.utils.to_categorical(y_train, num_classes=3)
test_yCl = tf.keras.utils.to_categorical(y_test, num_classes=3)
valid_yCl = tf.keras.utils.to_categorical(y_val, num_classes=3)

In [ ]:
resADD = tf.keras.applications.ResNet50V2(

    input_shape=(100, 100, 3),
    include_top = False

)
resADD.trainable = False

94668760/94668760 [==============================] - 1s 0us/step


In [ ]:
res_model = Sequential()

res_model.add(resADD)
res_model.add(Dropout(0.4))
res_model.add(MaxPooling2D())
res_model.add(Flatten())
res_model.add(Dense(1024, activation='relu'))
res_model.add(BatchNormalization())
res_model.add(Dropout(0.2))
res_model.add(Dense(3, activation='softmax'))

res_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
res_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50v2 (Functional)     (None, 4, 4, 2048)        23564800  
                                                                 
 dropout (Dropout)           (None, 4, 4, 2048)        0         
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 2, 2, 2048)        0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           (None, 8192)              0         
                                                                 
 dense (Dense)               (None, 1024)              8389632   
                                                                 
 batch_normalization (Batch  (None, 1024)              4096      
 Normalization)                                         

In [ ]:
from tensorflow import keras

callback_list = [
    keras.callbacks.ModelCheckpoint(
        filepath='model.h5',
        monitor = 'val_accuracy', save_best_only=True, verbose=3
    ),
    keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=3)
]

In [ ]:
history = res_model.fit(x_train, train_yCl,
                    batch_size=128,
                    validation_data = (x_val, valid_yCl),
                    callbacks = callback_list,
                    epochs = 13
                    )

Epoch 1/13
85/85 [==============================] - ETA: 0s - loss: 0.4888 - accuracy: 0.8259
Epoch 1: val_accuracy improved from -inf to 0.67917, saving model to model.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


85/85 [==============================] - 23s 165ms/step - loss: 0.4888 - accuracy: 0.8259 - val_loss: 1.3204 - val_accuracy: 0.6792
Epoch 2/13
84/85 [============================>.] - ETA: 0s - loss: 0.3340 - accuracy: 0.8649
Epoch 2: val_accuracy improved from 0.67917 to 0.69167, saving model to model.h5
85/85 [==============================] - 9s 107ms/step - loss: 0.3347 - accuracy: 0.8648 - val_loss: 0.8584 - val_accuracy: 0.6917
Epoch 3/13
84/85 [============================>.] - ETA: 0s - loss: 0.3115 - accuracy: 0.8716
Epoch 3: val_accuracy did not improve from 0.69167
85/85 [==============================] - 8s 96ms/step - loss: 0.3116 - accuracy: 0.8714 - val_loss: 2.0560 - val_accuracy: 0.5167
Epoch 4/13
84/85 [============================>.] - ETA: 0s - loss: 0.3007 - accuracy: 0.8780
Epoch 4: val_accuracy did not improve from 0.69167
85/85 [==============================] - 8s 99ms/step - loss: 0.3010 - accuracy: 0.8778 - val_loss: 1.0935 - val_accuracy: 0.6550
Epoch 5/13
8

In [ ]:
res_model = keras.models.load_model('/content/model.h5')

In [ ]:
score_train_res = res_model.evaluate(x_train, train_yCl)
print("Train Accuracy: ", score_train_res[1])

score_test_res = res_model.evaluate(x_test, test_yCl)
print("Test Accuracy: ", score_test_res[1])

score_val_res = res_model.evaluate(x_val, valid_yCl)
print("Validation Accuracy: ", score_val_res[1])

338/338 [==============================] - 17s 31ms/step - loss: 0.7246 - accuracy: 0.7259
Train Accuracy:  0.7259259223937988
94/94 [==============================] - 4s 41ms/step - loss: 0.7398 - accuracy: 0.7293
Test Accuracy:  0.7293333411216736
38/38 [==============================] - 1s 29ms/step - loss: 0.8584 - accuracy: 0.6917
Validation Accuracy:  0.6916666626930237


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

def rapor(model, x_test, test_yCl, categories):
    y_test_pred = model.predict(x_test)
    y_test_true = np.argmax(test_yCl, axis=1)
    y_test_pred = np.argmax(y_test_pred, axis=1)

    accuracy = accuracy_score(y_test_true, y_test_pred)
    precision = precision_score(y_test_true, y_test_pred, average='weighted')
    recall = recall_score(y_test_true, y_test_pred, average='weighted')
    f1 = f1_score(y_test_true, y_test_pred, average='weighted')

    conf_matrix = confusion_matrix(y_test_true, y_test_pred)
    class_report = classification_report(y_test_true, y_test_pred, target_names=categories)

    return accuracy, precision, recall, f1, conf_matrix, class_report


accuracy, precision, recall, f1, conf_matrix, class_report = rapor(res_model, x_test, test_yCl, CATEGORIES)
print(f"Test Accuracy: {accuracy}")
print(f"Test Precision: {precision:.4f}")
print(f"Test Recall: {recall}")
print(f"Test F1 Skoru: {f1:.4f}\n")

print("\nKarmaşıklık Matrisi:")
print(conf_matrix)

print("\nSınıflandırma Raporu:")
print(class_report)

94/94 [==============================] - 2s 26ms/step
Test Accuracy: 0.7293333333333333
Test Precision: 0.8219
Test Recall: 0.7293333333333333
Test F1 Skoru: 0.6677


Karmaşıklık Matrisi:
[[1035    2    0]
 [ 674  176  120]
 [  16    0  977]]

Sınıflandırma Raporu:
                              precision    recall  f1-score   support

lung_squamous cell carcinoma       0.60      1.00      0.75      1037
         lung_adenocarcinoma       0.99      0.18      0.31       970
          lung_benign tissue       0.89      0.98      0.93       993

                    accuracy                           0.73      3000
                   macro avg       0.83      0.72      0.66      3000
                weighted avg       0.82      0.73      0.67      3000

